In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Loading the Data**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Data_set = pd.read_csv("../input/amazon-top-50-bestselling-books-2009-2019/bestsellers with categories.csv")

**View summary of the data**

In [ ]:
Data_set.head()

In [ ]:
Data_set.describe()

**Checking to see if there is any correlation between number of reviews and ratings**

# Dropping duplicate books that have been republished over the years

In [ ]:
# sorting by Name 
Data_set.sort_values("Name", inplace = True) 
  
# dropping ALL duplicte values 
Data_set.drop_duplicates(subset ="Name", 
                     keep = False, inplace = True) 

In [ ]:
Data_set.describe()

Count has dropped from 550 to 255 showing that there were close to 50% of republished books

# Using Pairplot to see if there are any obvious correlation between variables

In [ ]:
# with regression
sns.pairplot(Data_set, kind="reg")
plt.show()

No apparent correlation appears between variables. We will now need to remove outliers and try to see if correlation exist in summarized data.

# Impact of Ratings on Number of Reviews

In [ ]:
y = list(Data_set['User Rating'])
x = list(Data_set['Reviews'])

plt.xlabel('Number of Reviews') 
plt.ylabel('Ratings') 
plt.title('Full Data Set View') 
plt.grid(True)

plt.scatter(x, y)
plt.show()

Turns out books with less than 4 ratings are hardly reviewed.
There are very few books with more than 30000 review

# **Removing Outliers**

In [ ]:
#redoing plot removing books with less than 4 rating or more than 30000 review

RefinedData_set = Data_set[((Data_set['User Rating'] >= 4) & (Data_set['Reviews'] <= 30000))]
y = list(RefinedData_set['User Rating'])
x = list(RefinedData_set['Reviews'])

plt.xlabel('Number of Reviews') 
plt.ylabel('Ratings') 
plt.title('Refind Data Set View') 
plt.grid(True) 

plt.scatter(x, y)
plt.show()

# Plotting to see the mean of number of reviews at different ratings 

In [ ]:
data = RefinedData_set.groupby('User Rating')['Reviews'].mean()
x = data.index.values
y = data.values

plt.xlabel('Ratings') 
plt.ylabel('Mean number of Reviews') 
plt.title('Ratings v/s Average Reviews') 
plt.grid(True) 

plt.plot(x, y)
plt.show()

# Impact of Price on Number of Reviews

In [ ]:
y = list(Data_set['Price'])
x = list(Data_set['Reviews'])

plt.xlabel('Number of Reviews') 
plt.ylabel('Price') 
plt.title('Full Data Set View') 
plt.grid(True)

plt.scatter(x, y)
plt.show()

# Removing Outliers

In [ ]:
#redoing plot removing books with Price > 40 or more than 40000 review

RefinedData_set = Data_set[((Data_set['Price'] <= 40) & (Data_set['Reviews'] <= 40000))]
y = list(RefinedData_set['Price'])
x = list(RefinedData_set['Reviews'])

plt.xlabel('Number of Reviews') 
plt.ylabel('Price') 
plt.title('Refined Data Set View') 
plt.grid(True) 

plt.scatter(x, y)
plt.show()

# Mean of Reviews Grouping By Price Range
Trying to group by price range and then taking mean of number of Reviews

In [ ]:
y = list(RefinedData_set['Price'])

step_count = 2
array_range = np.arange(0, max(y)+step_count, step_count)
data = RefinedData_set.groupby(pd.cut(RefinedData_set['Price'], array_range))['Reviews'].mean()

x = array_range[1:]
y = data.values

plt.xlabel('Price') 
plt.ylabel('Mean number of Reviews') 
plt.title('Price v/s Average Reviews') 
plt.grid(True) 

plt.plot(x, y)
plt.show()


# Impact of Price on Ratings

In [ ]:
y = list(Data_set['Price'])
x = list(Data_set['User Rating'])

plt.xlabel('User Rating') 
plt.ylabel('Price') 
plt.title('Full Data Set View') 
plt.grid(True)

plt.scatter(x, y)
plt.show()

To remove outliers will remove books with ratings < 4 and Price > 40

# Removing Outliers

In [ ]:
#redoing plot removing books with books with ratings < 4 and Price > 40

RefinedData_set = Data_set[((Data_set['Price'] <= 40) & (Data_set['User Rating'] >= 4))]
y = list(RefinedData_set['Price'])
x = list(RefinedData_set['User Rating'])

plt.xlabel('User Rating') 
plt.ylabel('Price') 
plt.title('Refined Data Set View') 
plt.grid(True) 

plt.scatter(x, y)
plt.show()

In [ ]:
y = list(RefinedData_set['Price'])

step_count = 2
array_range = np.arange(0, max(y)+step_count, step_count)
data = RefinedData_set.groupby(pd.cut(RefinedData_set['Price'], array_range))['User Rating'].mean()

x = array_range[1:]
y = data.values

plt.xlabel('Price') 
plt.ylabel('Mean User Rating') 
plt.title('Price v/s User Rating') 
plt.grid(True) 

plt.plot(x, y)
plt.show()


Clearly books in the 28 to 30 USD bracket have highest ratings

# Validating the range for highest ratings

In [ ]:
y = list(RefinedData_set['Price'])
step_count = 2
array_range = np.arange(0, max(y)+step_count, step_count)
data = RefinedData_set.groupby(pd.cut(RefinedData_set['Price'], array_range))[['User Rating','Reviews']].mean()

In [ ]:
data

It is clear that 28 to 30 is the range for highest ratings

# Ranking Books Based on Rating and Price. Which Book have the cheapest books at Highest Rating

In [ ]:
Revised_data = Data_set.groupby('Name')[['User Rating','Price']].mean()
Revised_data = Revised_data.sort_values(by = ['User Rating','Price'],ascending = [0,1])


Replace all 0 priced books with 0.1

In [ ]:
Revised_data['Price'] = Revised_data['Price'].replace(0,0.1)

From all the above attempt we see that even summarizing the data didnt help bring in any apparent correlation between these parameters

# Find the best Books to read in different price ranges 

like from 0 to 5 USD, 5 to 10 USD based on Ranking and Price

In [ ]:
Revised_data['Rating/Price'] = Revised_data['User Rating']/Revised_data['Price']
Revised_data = Revised_data.sort_values(by = ['Rating/Price'],ascending = [0])

step_count = 5
array_range = np.arange(0, max(y)+step_count, step_count)

In [ ]:
prev_price = 0
for i in array_range:
    if(i > 0):
        data = Revised_data[((Revised_data['Price'] > prev_price) & (Revised_data['Price'] <= i))]
        print ("\n" + str(prev_price) + " to " + str(i) + " : " )
        print(data.head(2))
        prev_price = i

In [ ]:
# 3D view of data
#y_z = data.values
#x_price = y_z[:,1]
#y_rating = y_z[:,0]
#z_review = array_range[1:]

# Data for three-dimensional scattered points
#ax = plt.axes(projection='3d')
#ax.scatter3D(x_price, y_rating, z_review, c=z_review, cmap='Greens')
#ax.plot3D(x_price, y_rating, z_review, 'gray')